# 🏡 Zurich Real Estate Price Prediction
## Exploratory Data Analysis (EDA)

Diese Notebook untersucht die Zürich Immobilienpreis-Datensätze und bereitet sie für die Modellierung vor.

In [ ]:
# Benötigte Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Anzeigeoptionen konfigurieren
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plot-Stil festlegen
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Daten laden

Wir laden die zwei Hauptdatensätze:
- `bau515od5155.csv`: Immobilienpreise nach Quartier
- `bau515od5156.csv`: Immobilienpreise nach Baualter

In [ ]:
# Datensätze laden
df_quartier = pd.read_csv('../data/raw/bau515od5155.csv', sep=',')
df_baualter = pd.read_csv('../data/raw/bau515od5156.csv', sep=',')

print(f"Quartier-Datensatz: {df_quartier.shape[0]} Zeilen, {df_quartier.shape[1]} Spalten")
print(f"Baualter-Datensatz: {df_baualter.shape[0]} Zeilen, {df_baualter.shape[1]} Spalten")

## 2. Datenübersicht

Zunächst verschaffen wir uns einen Überblick über die Struktur der Datensätze.

In [ ]:
# Quartier-Datensatz untersuchen
print("Quartier-Datensatz - Kopf:")
display(df_quartier.head())

print("\nQuartier-Datensatz - Info:")
df_quartier.info()

In [ ]:
# Baualter-Datensatz untersuchen
print("Baualter-Datensatz - Kopf:")
display(df_baualter.head())

print("\nBaualter-Datensatz - Info:")
df_baualter.info()

In [ ]:
# Statistische Zusammenfassung
print("Quartier-Datensatz - Statistik:")
display(df_quartier.describe())

print("\nBaualter-Datensatz - Statistik:")
display(df_baualter.describe())

## 3. Datenbereinigung

Wir reduzieren die Datensätze auf die relevanten Spalten und bereinigen sie.

In [ ]:
# Relevante Spalten auswählen - Quartier-Datensatz
quartier_spalten = {
    'Stichtagdatjahr': 'Jahr',              # Jahr der Datenerhebung
    'RaumLang': 'Quartier',                 # Name des Stadtquartiers 
    'AnzZimmerLevel2Lang_noDM': 'Zimmeranzahl',  # Zimmeranzahl als Text
    'HAMedianPreis': 'MedianPreis',         # Median-Verkaufspreis
    'HAPreisWohnflaeche': 'PreisProQm'      # Preis pro Quadratmeter
}

df_quartier_clean = df_quartier[quartier_spalten.keys()].copy()
df_quartier_clean.rename(columns=quartier_spalten, inplace=True)

# Relevante Spalten auswählen - Baualter-Datensatz
baualter_spalten = {
    'Stichtagdatjahr': 'Jahr',              # Jahr der Datenerhebung
    'BaualterLang_noDM': 'Baualter',        # Baualtersklasse als Text
    'AnzZimmerLevel2Lang_noDM': 'Zimmeranzahl',  # Zimmeranzahl als Text
    'HAMedianPreis': 'MedianPreis',         # Median-Verkaufspreis
    'HAPreisWohnflaeche': 'PreisProQm'      # Preis pro Quadratmeter
}

df_baualter_clean = df_baualter[baualter_spalten.keys()].copy()
df_baualter_clean.rename(columns=baualter_spalten, inplace=True)

# Bereinigte Datensätze anzeigen
print("Bereinigter Quartier-Datensatz:")
display(df_quartier_clean.head())

print("\nBereinigter Baualter-Datensatz:")
display(df_baualter_clean.head())

In [ ]:
# Fehlende Werte überprüfen
print("Fehlende Werte im Quartier-Datensatz:")
display(df_quartier_clean.isnull().sum())

print("\nFehlende Werte im Baualter-Datensatz:")
display(df_baualter_clean.isnull().sum())

## 4. Feature-Engineering

Wir extrahieren und transformieren Features aus den Textfeldern.

In [ ]:
# Zimmeranzahl von Text (z.B. "2-Zimmer") zu Zahl (2) konvertieren
def zimmer_zu_int(zimmer_str):
    """Extrahiert die Zimmeranzahl aus dem String-Format"""
    try:
        return int(zimmer_str.split('-')[0])
    except:
        return np.nan

df_quartier_clean['Zimmeranzahl_num'] = df_quartier_clean['Zimmeranzahl'].apply(zimmer_zu_int)
df_baualter_clean['Zimmeranzahl_num'] = df_baualter_clean['Zimmeranzahl'].apply(zimmer_zu_int)

# Baualter in numerisches Format umwandeln
def baualter_zu_jahr(baualter_str):
    """Wandelt Baualter-Text in ungefähres Baujahr um"""
    try:
        # Format "1981-2000"
        if '-' in baualter_str:
            jahre = baualter_str.split('-')
            return (int(jahre[0]) + int(jahre[1])) / 2
        # Format "vor 1919"
        elif 'vor' in baualter_str:
            return 1919
        # Format "nach 2015" oder "seit 2015"
        elif 'nach' in baualter_str or 'seit' in baualter_str:
            return 2015
        else:
            return np.nan
    except:
        return np.nan

df_baualter_clean['Baujahr'] = df_baualter_clean['Baualter'].apply(baualter_zu_jahr)

# Transformierte Daten anzeigen
print("Quartier-Datensatz mit numerischer Zimmeranzahl:")
display(df_quartier_clean.head())

print("\nBaualter-Datensatz mit numerischer Zimmeranzahl und Baujahr:")
display(df_baualter_clean.head())

## 5. Explorative Datenanalyse

Wir untersuchen die Verteilungen und Beziehungen in den Daten.

In [ ]:
# Verteilung der Immobilienpreise nach Quartier
plt.figure(figsize=(14, 8))
latest_year = df_quartier_clean['Jahr'].max()
df_latest = df_quartier_clean[df_quartier_clean['Jahr'] == latest_year]

# Nach Quartier gruppieren und Median-Preis berechnen
df_quartier_grouped = df_latest.groupby('Quartier')['MedianPreis'].median().sort_values(ascending=False).reset_index()

# Top 15 teuerste Quartiere visualisieren
top_15 = df_quartier_grouped.head(15)
sns.barplot(x='MedianPreis', y='Quartier', data=top_15)
plt.title(f'Top 15 teuerste Quartiere in Zürich ({latest_year})', fontsize=14)
plt.xlabel('Median-Preis (CHF)', fontsize=12)
plt.ylabel('Quartier', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Preisentwicklung über die Zeit für ausgewählte Quartiere
plt.figure(figsize=(14, 8))

# Einige repräsentative Quartiere auswählen
selected_quartiere = ['Hottingen', 'City', 'Seefeld', 'Oerlikon', 'Altstetten']

# Daten für ausgewählte Quartiere und 3-Zimmer-Wohnungen filtern
df_selected = df_quartier_clean[
    (df_quartier_clean['Quartier'].isin(selected_quartiere)) & 
    (df_quartier_clean['Zimmeranzahl_num'] == 3)
]

# Nach Jahr und Quartier gruppieren und Median-Preis berechnen
df_trend = df_selected.groupby(['Jahr', 'Quartier'])['MedianPreis'].median().reset_index()

# Liniendiagramm für Preisentwicklung
sns.lineplot(x='Jahr', y='MedianPreis', hue='Quartier', data=df_trend, marker='o')
plt.title('Preisentwicklung für 3-Zimmer-Wohnungen nach Quartier', fontsize=14)
plt.xlabel('Jahr', fontsize=12)
plt.ylabel('Median-Preis (CHF)', fontsize=12)
plt.grid(True)
plt.legend(title='Quartier')
plt.tight_layout()
plt.show()

In [ ]:
# Preis nach Zimmeranzahl und Baualter
plt.figure(figsize=(14, 8))

# Neueste Daten filtern
df_baualter_latest = df_baualter_clean[df_baualter_clean['Jahr'] == latest_year]

# Boxplot für Preis nach Zimmeranzahl
plt.subplot(1, 2, 1)
sns.boxplot(x='Zimmeranzahl_num', y='MedianPreis', data=df_baualter_latest)
plt.title(f'Preisverteilung nach Zimmeranzahl ({latest_year})', fontsize=14)
plt.xlabel('Anzahl Zimmer', fontsize=12)
plt.ylabel('Median-Preis (CHF)', fontsize=12)

# Streudiagramm für Preis vs. Baujahr
plt.subplot(1, 2, 2)
sns.scatterplot(x='Baujahr', y='MedianPreis', hue='Zimmeranzahl_num', data=df_baualter_latest)
plt.title(f'Preis vs. Baujahr ({latest_year})', fontsize=14)
plt.xlabel('Baujahr (geschätzt)', fontsize=12)
plt.ylabel('Median-Preis (CHF)', fontsize=12)
plt.legend(title='Zimmer')

plt.tight_layout()
plt.show()

## 6. Datensätze zusammenführen

Wir kombinieren die Informationen aus beiden Datensätzen für die Modellierung.

In [ ]:
# Aggregieren der Baualter-Daten nach Jahr und Zimmeranzahl
df_baualter_agg = df_baualter_clean.groupby(['Jahr', 'Zimmeranzahl_num']).agg({
    'MedianPreis': 'mean',
    'Baujahr': 'mean'
}).reset_index()

df_baualter_agg.rename(columns={
    'MedianPreis': 'MedianPreis_Baualter',
    'Baujahr': 'Durchschnitt_Baujahr'
}, inplace=True)

# Die aggregierten Baualter-Daten mit dem Quartier-Datensatz verbinden
df_merged = pd.merge(
    df_quartier_clean,
    df_baualter_agg,
    on=['Jahr', 'Zimmeranzahl_num'],
    how='left'
)

# Preisverhältnis berechnen: Quartierpreis zu durchschnittlichem Preis nach Baualter
df_merged['Preis_Verhältnis'] = df_merged['MedianPreis'] / df_merged['MedianPreis_Baualter']

# Zusammengeführten Datensatz anzeigen
display(df_merged.head())

## 7. Quartier-Preisniveau berechnen

Wir berechnen ein relatives Preisniveau für jedes Quartier.

In [ ]:
# Neustes Jahr für das finale Modelltraining wählen
neuestes_jahr = df_merged['Jahr'].max()
df_final = df_merged[df_merged['Jahr'] == neuestes_jahr].copy()

# Feature für Quartier-Preisniveau: Durchschnittlicher Preis im Quartier relativ zum Gesamtdurchschnitt
quartier_avg_preis = df_final.groupby('Quartier')['MedianPreis'].mean()
gesamtpreis_avg = quartier_avg_preis.mean()
quartier_preisniveau = (quartier_avg_preis / gesamtpreis_avg).to_dict()

df_final['Quartier_Preisniveau'] = df_final['Quartier'].map(quartier_preisniveau)

# Preisniveau-Ranking anzeigen
quartier_ranking = pd.DataFrame({
    'Quartier': quartier_preisniveau.keys(),
    'Preisniveau': quartier_preisniveau.values()
}).sort_values('Preisniveau', ascending=False).reset_index(drop=True)

display(quartier_ranking.head(10))

In [ ]:
# Visualisierung des Preisniveaus nach Quartier
plt.figure(figsize=(14, 8))

# Top 15 Quartiere nach Preisniveau
top_15_preisniveau = quartier_ranking.head(15)
sns.barplot(x='Preisniveau', y='Quartier', data=top_15_preisniveau)
plt.axvline(x=1.0, color='red', linestyle='--', label='Durchschnitt')
plt.title('Top 15 Quartiere nach relativem Preisniveau', fontsize=14)
plt.xlabel('Relatives Preisniveau (1.0 = Durchschnitt)', fontsize=12)
plt.ylabel('Quartier', fontsize=12)
plt.legend()
plt.tight_layout()
plt.show()

## 8. Vorbereitung der Modelldaten

Wir bereiten den finalen Datensatz für das Modelltraining vor.

In [ ]:
# Kategorische Features vorbereiten (für ML-Modelle wie Random Forest)
# Quartier als kategorisches Feature - später One-Hot-Encoding anwenden
df_final['Quartier_Code'] = pd.Categorical(df_final['Quartier']).codes

# Fehlende Werte behandeln
df_final.dropna(subset=['MedianPreis', 'Quartier', 'Zimmeranzahl_num'], inplace=True)

# Restliche NaN-Werte durch sinnvolle Werte ersetzen
for column in df_final.columns:
    if df_final[column].dtype in [np.float64, np.int64]:
        df_final[column].fillna(df_final[column].median(), inplace=True)

# Finalen Datensatz anzeigen
print(f"Finaler Datensatz für das Modelltraining: {df_final.shape[0]} Zeilen, {df_final.shape[1]} Spalten")
display(df_final.head())

In [ ]:
# Korrelationsmatrix berechnen und visualisieren
plt.figure(figsize=(14, 10))

# Nur numerische Spalten auswählen
numeric_columns = df_final.select_dtypes(include=['float64', 'int64']).columns
correlation_matrix = df_final[numeric_columns].corr()

# Heatmap erstellen
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Korrelationsmatrix der numerischen Features', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Daten speichern

Wir speichern die verarbeiteten Datensätze für die weitere Verwendung.

In [ ]:
# Verarbeitete Datensätze speichern
df_quartier_clean.to_csv('../data/processed/quartier_processed.csv', index=False)
df_baualter_clean.to_csv('../data/processed/baualter_processed.csv', index=False)
df_final.to_csv('../data/processed/modell_input_final.csv', index=False)

print("Datensätze wurden erfolgreich gespeichert!")

## 10. Zusammenfassung und nächste Schritte

**Zusammenfassung:**
- Die Immobilienpreise variieren stark zwischen den verschiedenen Quartieren in Zürich
- Die teuersten Quartiere sind: Seefeld, City, Hottingen
- Es gibt einen klaren positiven Zusammenhang zwischen Zimmeranzahl und Preis
- Neuere Gebäude sind tendenziell teurer als ältere
- Die Preise sind über die Jahre kontinuierlich gestiegen

**Nächste Schritte:**
1. Reisezeit-Daten generieren und integrieren
2. ML-Modelle (Random Forest, Gradient Boosting) trainieren
3. Modelle evaluieren und optimieren
4. Streamlit-App entwickeln mit interaktiven Karten
5. Video-Präsentation erstellen